# AIM 5001 Week 10 Assignment : Tidying and Transforming Data

## Alejandra Zapata

In [225]:
import pandas as pd

### Task 1: Create CSV
* Load CSV from GitHub repository

In [226]:
flights = pd.read_csv("https://raw.githubusercontent.com/AlejandraZS10/W10-assn-AIM-5001/master/A_Zapata_W10_assn1.csv")
flights

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497,221,212,503,1841
1,NaN,delayed,62,12,20,102,305
2,AM WEST,on time,694,4840,383,320,201
3,NaN,delayed,117,415,65,129,61


### Task 2: 'Long' Format
* To translate to long format I will create four columns:
    * Airline
    * Destinations
    * On time arrivals
    * Delayed arrivals

In [227]:
#extract destinations and time to avoid the NaN values
dest = flights.iloc[:, 2:]
dest

,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,497,221,212,503,1841
1,62,12,20,102,305
2,694,4840,383,320,201
3,117,415,65,129,61


In [228]:
#we will use the keys to add the airline and performance values
#use stack to create long format
long_dest = dest.stack()
#transform to dataframe
long = pd.DataFrame(long_dest)
#change obj to int type for analysis purposes
long[0] = long[0].astype(int)
#print long format of city and times
long

0
0 Los Angeles     497
  Phoenix         221
  San Diego       212
  San Francisco   503
  Seattle        1841
1 Los Angeles      62
  Phoenix          12
  San Diego        20
  San Francisco   102
  Seattle         305
2 Los Angeles     694
  Phoenix        4840
  San Diego       383
  San Francisco   320
  Seattle         201
3 Los Angeles     117
  Phoenix         415
  San Diego        65
  San Francisco   129
  Seattle          61

In [229]:
#use indexes to assign on time and delayed values
long_flights = long.rename(index={0: 'on time',1: 'delayed', 2: 'on time', 3: 'delayed'})
#convert to dataframe
long_flights2 = pd.DataFrame(long_flights)
#reset indexes
long_flights2.reset_index(inplace=True)
#print
long_flights2

,level_0,level_1,0
0,on time,Los Angeles,497
1,on time,Phoenix,221
2,on time,San Diego,212
3,on time,San Francisco,503
4,on time,Seattle,1841
5,delayed,Los Angeles,62
6,delayed,Phoenix,12
7,delayed,San Diego,20
8,delayed,San Francisco,102
9,delayed,Seattle,305


In [230]:
#rename the columns and assign index values the airlines
long_flights3 = long_flights2.rename(columns={'level_0': 'Performance','level_1': 'Destination', 0 :'Time'}
                         ,index={0: 'ALASKA', 1: 'ALASKA',2: 'ALASKA', 3: 'ALASKA', 4: 'ALASKA', 5: 'ALASKA', 
                                 6: 'ALASKA',7: 'ALASKA', 8: 'ALASKA', 9: 'ALASKA',
                             10: 'AM WEST', 11: 'AM WEST', 12: 'AM WEST', 13: 'AM WEST', 14: 'AM WEST', 
                                15: 'AM WEST',16: 'AM WEST', 17: 'AM WEST', 18: 'AM WEST', 19: 'AM WEST'})
#reset the index
long_flights4 = long_flights3.reset_index()
#rename the column to airline
long_flights5 = long_flights4.rename(columns={'index': 'Airline'})
#print our long format data frame
long_flights5

,Airline,Performance,Destination,Time
0,ALASKA,on time,Los Angeles,497
1,ALASKA,on time,Phoenix,221
2,ALASKA,on time,San Diego,212
3,ALASKA,on time,San Francisco,503
4,ALASKA,on time,Seattle,1841
5,ALASKA,delayed,Los Angeles,62
6,ALASKA,delayed,Phoenix,12
7,ALASKA,delayed,San Diego,20
8,ALASKA,delayed,San Francisco,102
9,ALASKA,delayed,Seattle,305


* The long format has a single observation per row in a total of 19 and 4 columns 

### Task 3: Answer questions using our long format
* Extract only on time 
* Use aggregate functions and group by to determine the answers to the questions

#### 3.1. For each city, which airline had the best on time performance?

In [231]:
long_flights6=long_flights5.loc[long_flights5['Performance'] == 'on time']
#show on time data frame
long_flights6

,Airline,Performance,Destination,Time
0,ALASKA,on time,Los Angeles,497
1,ALASKA,on time,Phoenix,221
2,ALASKA,on time,San Diego,212
3,ALASKA,on time,San Francisco,503
4,ALASKA,on time,Seattle,1841
10,AM WEST,on time,Los Angeles,694
11,AM WEST,on time,Phoenix,4840
12,AM WEST,on time,San Diego,383
13,AM WEST,on time,San Francisco,320
14,AM WEST,on time,Seattle,201


In [232]:
#create group by of destination
grouped = long_flights6.groupby('Destination')
#extract max values per city
maximums = grouped.max()
#reset index to match the original format
maximums.reset_index()

,Destination,Airline,Performance,Time
0,Los Angeles,AM WEST,on time,694
1,Phoenix,AM WEST,on time,4840
2,San Diego,AM WEST,on time,383
3,San Francisco,AM WEST,on time,503
4,Seattle,AM WEST,on time,1841


In [233]:
#use group by and max on one line 
long_flights6.groupby(['Destination'])['Time'].max().reset_index()

,Destination,Time
0,Los Angeles,694
1,Phoenix,4840
2,San Diego,383
3,San Francisco,503
4,Seattle,1841


* For Los Angeles, Phoenix, San Diego AM WEST had the best on time and for San Francisco and Seattle ALASKA had the max on time performance.

#### 3.2. Which airline had the best overall on time performance?

In [234]:
# use group by and mean on one line to obtain the overall performance per airline
long_flights6.groupby(['Airline'])['Time'].mean().reset_index()

,Airline,Time
0,ALASKA,654.8
1,AM WEST,1287.6


* AM WEST had a better overall on time performance

### Task 4: Change to 'wide' format
* Use to unstack to change from 'long' to 'wide' format.

In [235]:
#unstack and .DataFrame on one line to obtain the right structure
pd.DataFrame(df5.unstack().unstack())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Airline,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,ALASKA,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST,AM WEST
Performance,on time,on time,on time,on time,on time,delayed,delayed,delayed,delayed,delayed,on time,on time,on time,on time,on time,delayed,delayed,delayed,delayed,delayed
Destination,Los Angeles,Phoenix,San Diego,San Francisco,Seattle,Los Angeles,Phoenix,San Diego,San Francisco,Seattle,Los Angeles,Phoenix,San Diego,San Francisco,Seattle,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
Time,497,221,212,503,1841,62,12,20,102,305,694,4840,383,320,201,117,415,65,129,61
